In [ ]:
import cv2
import torch
import 
import numpy as np

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#Load model and weights
model =                           # your model class
state = torch.load("weights.pth", map_location=DEVICE)
model.load_state_dict(state)
model.to(DEVICE)
model.eval()                                      # important: inference mode

NameError: name 'MyFaceDetector' is not defined

In [ ]:
def preprocess(frame, size=(224, 224)):
    # OpenCV frame is BGR uint8 (H, W, 3)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    resized = cv2.resize(rgb, size)               # (H, W, 3)
    x = resized.astype(np.float32) / 255.0        # scale 0..1

    # HWC -> CHW
    x = np.transpose(x, (2, 0, 1))                # (3, H, W)

    # Add batch dimension: (3,H,W)->(1,3,H,W)
    x = np.expand_dims(x, axis=0)

    # NumPy -> Torch tensor
    x = torch.from_numpy(x).to(DEVICE)            # float32 tensor on device
    return x

In [ ]:
def predict_boxes(frame):
    x = preprocess(frame)

    # 2) Run the model
    output = model(x)

    # 3) Convert output to boxes in original frame coordinates
    # This depends on your model. Placeholder format:
    # boxes = [(x1, y1, x2, y2, score), ...]
    boxes = decode_output_to_boxes(output, frame.shape)

    return boxes

In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Could not open webcam")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        boxes = predict_boxes(frame)

        # draw boxes
        for (x1, y1, x2, y2, score) in boxes:
            cv2.rectangle(frame, (x1, y1), (x2, y2), 2)
            cv2.putText(frame, f"{score:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)

        cv2.imshow("Face Detector", frame)
        if (cv2.waitKey(1) & 0xFF) == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    main()